In [51]:
# 데이터 베이스 연동 후 -> 수집한 데이터를 DB에 저장 (csv)
import pymysql, pass1, time, re
from datetime import datetime, timedelta

conn = pymysql.connect(
    host='localhost',
    user='root',
    password= pass1.password(),
    db = 'booksminiproject',
    cursorclass = pymysql.cursors.DictCursor
)

# 특정 날짜 설정, 예: '2024-02-10'
yesterday = datetime.now() - timedelta(days=1)
# 어제 날짜를 'YYYY-MM-DD' 형식의 문자열로 변환
yesterday_str = yesterday.strftime('%Y-%m-%d')

# specific_date = datetime(2024, 2, 10)
specific_date = datetime.now().strftime('%Y-%m-%d')

try:
    with conn.cursor() as cursor:
        # ISBN 목록 가져오기
        cursor.execute(f"SELECT books.isbn FROM books join kyobo_ranking on books.isbn = kyobo_ranking.isbn join yes24_ranking on books.isbn = yes24_ranking.isbn where kyobo_ranking.inputdate = '{specific_date}' and yes24_ranking.inputdate = '{specific_date}'")
        isbns = cursor.fetchall()


        for isbn_record in isbns:
            isbn = isbn_record['isbn']

            # kyobo와 yes24의 평점과 순위 데이터 가져오기 (특정 날짜에 대해서만)
            cursor.execute("""
            SELECT 
                (SELECT AVG(kyoboRating) FROM kyobo_ranking WHERE isbn = %s AND inputDate = %s) as avgKyoboRating, 
                (SELECT AVG(yes24Rating) FROM yes24_ranking WHERE isbn = %s AND inputDate = %s) as avgYes24Rating,
                (SELECT AVG(kyoboRank) FROM kyobo_ranking WHERE isbn = %s AND inputDate = %s) as avgKyoboRank, 
                (SELECT AVG(yes24Rank) FROM yes24_ranking WHERE isbn = %s AND inputDate = %s) as avgYes24Rank
            """, (isbn, specific_date, isbn, specific_date, isbn, specific_date, isbn, specific_date))

            avg_data = cursor.fetchone()

            cursor.execute(f"SELECT averageWeekRanking FROM average WHERE inputdate = '{specific_date}'  and isbn = '{isbn}'")
            yesterday_num = cursor.fetchone()

            if yesterday_num != None:
                averageWeekRanking = yesterday_num['averageWeekRanking'] + 1
            else:
                averageWeekRanking = 1


            if avg_data and any(avg_data.values()):  # 데이터가 있고, 모든 값이 None이 아닐 때만 처리
                averageRating = (avg_data['avgKyoboRating'] or 0 + avg_data['avgYes24Rating'] or 0) / 2
                averageRanking = (avg_data['avgKyoboRank'] or 0 + avg_data['avgYes24Rank'] or 0) / 2
                
                # averageWeekRanking 계산 로직은 생략되었으나, 필요에 따라 추가할 수 있음
                # average 테이블에 데이터 삽입
                cursor.execute("""
                INSERT INTO average (isbn, inputDate, averageRating, averageRanking, averageWeekRanking)
                VALUES (%s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    averageRating = VALUES(averageRating),
                    averageRanking = VALUES(averageRanking)
                """, (isbn, specific_date, averageRating, averageRanking, averageWeekRanking))

                conn.commit()

finally:
    conn.close()


In [19]:
import pymysql

# 데이터베이스 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password= pass1.password(),
    db = 'booksminiproject',
    cursorclass = pymysql.cursors.DictCursor
    )

try:
    with conn.cursor() as cursor:
        # 현재 데이터베이스의 모든 테이블 목록 조회
        cursor.execute("SHOW TABLES")
        tables = cursor.fetchall()
        print(tables)
        for table_name in tables:
            print(f"Table: {table_name['Tables_in_booksminiproject']}")
            # 각 테이블에 대해 DESCRIBE 명령 실행
            cursor.execute(f"DESCRIBE {table_name['Tables_in_booksminiproject']}")
            for row in cursor.fetchall():
                print(row)
            print("\n")  # 테이블 간 구분을 위해 빈 줄 추가
finally:
    conn.close()


[{'Tables_in_booksminiproject': 'alembic_version'}, {'Tables_in_booksminiproject': 'average'}, {'Tables_in_booksminiproject': 'books'}, {'Tables_in_booksminiproject': 'kyobo_price'}, {'Tables_in_booksminiproject': 'kyobo_ranking'}, {'Tables_in_booksminiproject': 'yes24_price'}, {'Tables_in_booksminiproject': 'yes24_ranking'}]
Table: alembic_version
{'Field': 'version_num', 'Type': 'varchar(32)', 'Null': 'NO', 'Key': 'PRI', 'Default': None, 'Extra': ''}


Table: average
{'Field': 'priceId', 'Type': 'int', 'Null': 'NO', 'Key': 'PRI', 'Default': None, 'Extra': 'auto_increment'}
{'Field': 'isbn', 'Type': 'varchar(13)', 'Null': 'NO', 'Key': 'MUL', 'Default': None, 'Extra': ''}
{'Field': 'inputDate', 'Type': 'date', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}
{'Field': 'averageRating', 'Type': 'decimal(3,1)', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}
{'Field': 'averageRanking', 'Type': 'int', 'Null': 'YES', 'Key': '', 'Default': None, 'Extra': ''}
{'Field': 'average